# One target island (openCV 3.4.2)

## Import modules

Make sure that Python 3 and the following modules (recommended version ID) are installed on your computer before running this cell:

numpy (1.18.1),
sounddevice (0.3.14),
openCV (3.4.2),
tkinter (8.6.8),
xlxswriter (1.2.7),
scipy (1.3.2),
pyfirmata (1.1.0)

In [3]:
import numpy as np                        # Import numpy module
                                          # conda install -c conda-forge python-sounddevice
import sounddevice as sd                  # Import sounddevice module for "real-time" sound playback
import cv2                                # Import opencv module for image processing
print(cv2.__version__)                    # Make sure you have the correct version of openCV 
import tkinter as tk                      # Import Tkinter module for GUIs
import xlsxwriter                         # Import xlsxwriter module for writing a protocol on-the-fly

from scipy.io  import wavfile             # WAV-file import filter
from pyfirmata import Arduino             # Arduino support
from math      import e                   # Euler's number

import math                               # Import math module
import time                               # Import time module for time measurements and pausing
import random                             # Import random module for random number generation


4.2.0


## Define a custom function

In [4]:
# Function that defines global variables based on user inputs at a starting screen (below)
def show_entry_fields():
    
    print('Trials per Session: %s' % e1.get())
    print('Session Duration: %s' % e2.get())
    print('Trial Duration: %s' % e3.get())
    print('Radius of the Starting Platform: %s' % e4.get())
    print('X-Coordinate of the Starting Platform: %s' % e5.get())
    print('Y-Coordinate of the Starting Platform: %s' % e6.get())
    print('Radius of the target platform: %s' % e7.get())
    print('Target duration: %s' % e8.get())
    print('Subject and Date: %s' % e9.get())
    print('Subject is darker than background: %s' % e10.get())
    print('Initialization Duration: %s' % e11.get())
    
    global trialNumber 
    trialNumber = int(e1.get())
    global sessionDuration 
    sessionDuration = int(e2.get())
    global trialDuration 
    trialDuration = int(e3.get())
    global startRadius
    startRadius = int(e4.get())
    global startX
    startX = int(e5.get())
    global startY
    startY = int(e6.get())
    global targetRadius
    targetRadius = int(e7.get())
    global targetDuration
    targetDuration = float(e8.get())
    global experimentID
    experimentID = e9.get()
    global backgroundColor
    backgroundColor = e10.get()
    global initDuration
    initDuration = float(e11.get())


## Set the arena dimensions

Here, the arena position and dimensions can be defined. In the example setup, a circular arena of 80 cm radius is filmed by a USB-webcam with a resolution of 800*600 px. In the cell below, the arena dimensions optimally use the resolution of the webcam in terms of spatial resolution of the later tracking. You may have to redefine the values in the next cell to fit your combination of camera resolution and arena dimensions.

In [5]:
# Arena coordinates and radius (best set to maximum spatial resolution at given image size)
# Here: Video resolution = 800*600
arenaX      = 400
arenaY      = 300
arenaRadius = 300

## Open a starting screen

The next cell will open a window that can be used to configure the experiment by entering the desired number of trials per session (usually limited by the number of rewards the feeder provides without refil), the session and trial durations, the size (here: 134 px in diameter = 17.87 cm) and the XY-position of the starting platform, the size of the target area, and the duration the animal has to spend in the target to recieve a reward. In addition, you can give your experiment a purposeful ID (e.g. subject ID, experiment type and date), provide information about the contrast between arena and subject, and define the duration the animal has to spend in the starting area to initialize a new trial. 

If you choose your target size, always make sure that it is small enough to fit in the arena without overlap with the starting/initialization area. 

The experiment ID you enter in the popup window will automatically be added to the file names of the protocols that will be generated for each session.

**To save your configuration, hit the apply button**. To close the popup window and proceed, hit the continue button.

In [8]:
# Starting screen
master = tk.Tk()
master.title('Experimental parameters')
tk.Label(master, text="Instructions: \n 1. Enter parameters (only integers are allowed as numbers) \n 2. Press 'Apply' \n 3. Press 'Continue'").grid(row=0, padx=10, pady=10)
tk.Label(master, text="Trials per session").grid(row=4, padx=5, pady=5)
tk.Label(master, text="Session duration [s]").grid(row=5, padx=5, pady=5)
tk.Label(master, text="Trial duration [s]").grid(row=6, padx=5, pady=5)
tk.Label(master, text="Radius of the starting platform [pixels]").grid(row=7, padx=5, pady=5)
tk.Label(master, text="X-position of the starting platform [pixels]").grid(row=8, padx=5, pady=5)
tk.Label(master, text="Y-position of the starting platform [pixels]").grid(row=9, padx=5, pady=5)
tk.Label(master, text="Radius of the target platform [pixels]").grid(row=10, padx=5, pady=5)
tk.Label(master, text="Target duration [s]").grid(row=11, padx=5, pady=5)
tk.Label(master, text="Experiment ID [subject_date]").grid(row=12, padx=5, pady=5)
tk.Label(master, text="Subject is darker than background [T = True; F = False]").grid(row=13, padx=5, pady=5)
tk.Label(master, text="Initialisation Duration [s]").grid(row=14, padx=5, pady=5)

e1 = tk.Entry(master)
e1.insert('end', '50')
e2 = tk.Entry(master)
e2.insert('end', '3600')
e3 = tk.Entry(master)
e3.insert('end', '60')
e4 = tk.Entry(master)
e4.insert('end', '67')
e5 = tk.Entry(master)
e5.insert('end', '195')
e6 = tk.Entry(master)
e6.insert('end', '195')
e7 = tk.Entry(master)
e7.insert('end', '80')
e8 = tk.Entry(master)
e8.insert('end', '5')
e9 = tk.Entry(master)
e9.insert('end', 'SubjectID_ExpType_MM_DD_20YY')
e10 = tk.Entry(master)
e10.insert('end', 'T')
e11 = tk.Entry(master)
e11.insert('end', '0.2')

e1.grid(row=4, column=1)
e2.grid(row=5, column=1)
e3.grid(row=6, column=1)
e4.grid(row=7, column=1)
e5.grid(row=8, column=1)
e6.grid(row=9, column=1)
e7.grid(row=10, column=1)
e8.grid(row=11, column=1)
e9.grid(row=12, column=1)
e10.grid(row=13, column=1)
e11.grid(row=14, column=1)

tk.Button(master, text='Apply', command=show_entry_fields).grid(row=15, column=0, sticky='s', pady=4)
tk.Button(master, text='Continue', command=master.destroy).grid(row=15, column=1, sticky='w', pady=4)

tk.mainloop()



Trials per Session: 50
Session Duration: 3600
Trial Duration: 60
Radius of the Starting Platform: 67
X-Coordinate of the Starting Platform: 195
Y-Coordinate of the Starting Platform: 195
Radius of the target platform: 80
Target duration: 5
Subject and Date: SubjectID_ExpType_MM_DD_20YY
Subject is darker than background: T
Initialization Duration: 0.2


## Protocol 1

Run the upcoming cell, if you want to save the chosen experimetal parameters to a txt-file ("ExperimentID_parameters.txt"). The file will be saved to the folder containing this notebook.

In [9]:
# Saves all parameters to a txt-file with the user-defined "Experiment ID" as filename
parametersName = experimentID + '_parameters.txt'
with open(parametersName, 'w') as f:
    print(time.asctime(time.localtime(time.time())), file=f)
    print('Trials per Session: %s' % trialNumber, file=f)
    print('Session Duration: %s' % sessionDuration, file=f)
    print('Trial Duration: %s' % trialDuration, file=f)
    print('Radius of the Starting Platform: %s' % startRadius, file=f)
    print('X-Coordinate of the Starting Platform: %s' % startX, file=f)
    print('Y-Coordinate of the Starting Platform: %s' % startY, file=f)
    print('Radius of the target platform: %s' % targetRadius, file=f)
    print('Duration the subject has to stay in the target area: %s' % targetDuration, file=f)
    print('Subject and Date: %s' % experimentID, file=f)
    print('Subject is darker than background: %s' % backgroundColor, file=f)
    print('Initialization Duration: %s' % initDuration, file=f)


## Initialize the microcontroller

The next cell Initializes a microcontroller for subsequent hardware control. This is, where you will probably have to get creative yourself, depending on what you would like to do. Here, we use an Arduino Nano. With the channel definitions below, we can later provide differently colored illumination during the experiment (for example to stimulate with colors rather than sound) and trigger two different feeders. 

For the example setup, two automatic fish feeders with 27 feeding slots each were "hacked", so that they can be controlled *via* two additional Arduinos with motor shields. These additional Arduinos drive the feeder motors each time they get a trigger signal from the main Arduino. The two feeders allow the provision of 54 rewards per session. The two feeders were installed at different positions above the arena and are activated alternately, to lower the predictability of where in the arena the reward will drop. The starting feeder is chosen randomly for each new session.

In [ ]:
# Define colors and feeder channel for Arduino output
arduinoBlue = 9         # Blue diodes
arduinoYellow = 10      # Yellow diodes
arduinoRed = 11         # Red diodes
arduinoFeeder1 = 12     # Trigger pulse for feeder1
arduinoFeeder2 = 4      # Trigger pulse for feeder2

# Feeder changes every trial, start feeder randomized
feederID = random.randrange(1,3,1)

# Initialize Arduino
board = Arduino('COM3')    # May be another COM-Port - in Windows, just check the Hardware Manager

## Prepare the audio stream

The following cell initiates the audio stream, to which we will later feed our stimuli. The default sample rate is set to 44.1 kHz. The cell also loads sound files with the stimuli. Here, we use short pure tones as stimuli and a silent sound object, which is fed to the audiostream between stimuli. In our setup, we found this to be necessarry to reduce undesired clicking sounds at stimulus on- and offset, even though the sounds are ramped. Whether this will be necessary for you, will strongly depend on your audio hardware. 

The audio stimulation provided by this notebook differs from the MATLAB version in two important aspects: Firstly, the MATLAB version generates the stimuli on the fly, while this notebook uses sound files as input. Feel free to change the code if you prefer the other solution. Secondly, the MATLAB version stimulates at fixed time intervals and the sample rate of the video tracking is locked to the stimulation interval, i.e. high temporal precision in the sound stimulation comes with the cost of lower temporal resolution of the animal tracking. Here, we chose the opposite approach, with the video feed defining the cycle frequency (approx. 14 Hz with the given Camera and a resolution of 800x600 px) and the audio stimulation being locked to the framerate of the camera. Thus, higher temporal resolution of the animal tracking comes with the cost that inter-stimulus intervals cannot freely be chosen, but only be multiple integers (3 or higher) of the mean video frame duration. In the example setup and the code below, we decided for the stimulus to be played every three cycles (approx. every 215 ms). 

The duration of the audio files should not exceed the cycle length.

In [11]:
# Set sample rate for audio output
sd.default.samplerate = 44100
fs = 44100          

# Audio stream
stream = sd.OutputStream(samplerate=fs, channels=1, dtype='float32')

# Cycle counter: sound is played every "delayLength" cycles
commonCycle = 1
delayLength = 3  

# Open sound files
distractorSoundTrial = wavfile.read('./10kHz-short-68.wav')[1]
attractorSoundTarget1 = wavfile.read('./4000Hz-short-68.wav')[1]
silenceSound = wavfile.read('./silence-short-68.wav')[1]
  

<ipython-input-11-1969cbc2e2a8>:13: WavFileWarning: Chunk (non-data) not understood, skipping it.
  distractorSoundTrial = wavfile.read('./10kHz-short-68.wav')[1]
<ipython-input-11-1969cbc2e2a8>:14: WavFileWarning: Chunk (non-data) not understood, skipping it.
  attractorSoundTarget1 = wavfile.read('./4000Hz-short-68.wav')[1]
<ipython-input-11-1969cbc2e2a8>:15: WavFileWarning: Chunk (non-data) not understood, skipping it.
  silenceSound = wavfile.read('./silence-short-68.wav')[1]


In [12]:
silenceSound


array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

## Protocol 2

The following cell generates a video object to which the later video feed will be saved. The colours that are defined will later be used for labeling. The labelled video file ("ExperimentID_video.avi") will be saved to the folder containing this notebook for documentation purposes. 

In [ ]:
# Define BGR colors
BGR_COLOR = {'red': (0,0,255),
             'green': (127,255,0),
             'blue': (255,127,0),
             'yellow': (0,127,255),
             'black': (0,0,0),
             'white': (255,255,255)}

# Define the codec and create VideoWriter object
videoName = experimentID + '_video.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
# Make sure that the frame rate of your output appoximately matches 
# the number of cycles per second, to avoid time lapsed output videos
out = cv2.VideoWriter(videoName,fourcc, 15.0, (800,600))


## Capture a background image

The tracking algorithm used in this notebook compares the frames of the video feed during the experiment with an image of the empty arena to later track the position of the largest object in the arena (which usually is your animal). If you are confident in the stability of your video quality, it should suffice to capture the picture once and to skip this cell in the subsequent experiments. However, since this step only takes a few seconds, we recommend to take a new picture of the arena for each new experiment. In the preview of the video feed that will pop-up if you run the next cell, the space outside the arena is masked, so that the camera preview can also be used to check if the camera/arena are still positioned correctly. 

Before taking the picture, make sure that the conditions in your lab (especially the illumination) are the exact same as they will be during the experiments. Once you are happy with the preview of your background image, press "c" to capture the image. It will be saved as "Background.png" to the folder containing this notebook.

This notebook will use the main camera of your system as an input device. If you have more than one camera installed (e.g. on a notebook with internal chat camera), make sure to deactivate all cameras other than the camera of your setup  prior to running the notebook. Also make sure that the video dimensions defined here match you arena dimensions defined above and the video dimensions of the video feeds that will be defined in the subsequent cells.

In [ ]:
# Define video capture device (0 = webcam1) to capture background frame
cap = cv2.VideoCapture(0)
# Set picture dimensions
cap.set(3,800)      # Width
cap.set(4,600)      # Height

# Capture Background frame (c = capture)
while(True):
    # Capture frame-by-frame
    ret, img = cap.read()
    img2 = img

    # Display the resulting frame
    imgArena = cv2.circle(img,(arenaX,arenaY), arenaRadius, (0,0,255), 2)
    imgArenaStart = cv2.circle(imgArena,(startX,startY), startRadius, (255,0,255), 2)

    # Mask the space outside the arena
    mask = np.zeros(shape = img.shape, dtype = "uint8")
    cv2.circle(mask, (arenaX,arenaY), arenaRadius, (255,255,255), -1)

    maskedImg2 = cv2.bitwise_and(src1 = img2, src2 = mask)
    imgArenaStart = cv2.bitwise_and(src1 = imgArenaStart, src2 = mask)

    cv2.imshow('Press (c)-to capture the background image',imgArenaStart)
    if cv2.waitKey(1) & 0xFF == ord('c'):
        cv2.imwrite('Background.png',maskedImg2)
        break

# When the background image is captured, release the capture
cap.release()
cv2.destroyAllWindows()

# Loads current background as object img for later use
img = cv2.imread('Background.png',1)

## Prepare the experiment

The following cell will provide another preview of the video feed from the arena. It will allow you to double-check if everything is prepared for the experiment. If so, you can bring your animal and put it into the arena. 

Once you have left the room with your setup and are happy with what you see in the live-feed, hit "c" to close the preview.

In [ ]:
# Define video capture device for live-stream (0 = webcam1)
cap2 = cv2.VideoCapture(0)
# Set picture dimensions
cap2.set(3,800)
cap2.set(4,600)

# Show video to see animal leaving the box
while(True):
    # Capture frame-by-frame
    ret, img3 = cap2.read()
    
    cv2.imshow('Press (c)-to continue',img3)
    if cv2.waitKey(1) & 0xFF == ord('c'):
        break

cap2.release()
cv2.destroyAllWindows()


## Initialize the camera

This cell initializes the camera for the actual tracking and defines some counters and dummy variables needed during the experiment.

In [ ]:
# Define video capture device for live-stream (0 = webcam1) and tracking
cap = cv2.VideoCapture(0)
# Set picture dimensions
cap.set(3,800)
cap.set(4,600)

# Mask the space outside the arena
mask = np.zeros(shape = img.shape, dtype = "uint8")
cv2.circle(mask, (arenaX,arenaY), arenaRadius, (255,255,255), -1)

# Experiment starts in phase 0 with 0 trials
expPhase = 0   
trialCounter = 0
rewardCounter = 0
frameCounter = 0
trialCountdown = 0
targetCountdown = 0

# Dummy values for target area generation (up to 5)
randomX = 9000     
randomY = 9000     
random2X = 9000     
random2Y = 9000     
random3X = 9000     
random3Y = 9000     
random4X = 9000     
random4Y = 9000     
random5X = 9000     
random5Y = 9000     
targetX = 9999
targetY = 9999
target2X = 9999
target2Y = 9999
target3X = 9999
target3Y = 9999
target4X = 9999
target4Y = 9999
target5X = 9999
target5Y = 9999


## Protocol 3

The following cell generates an Excel-file to which the essential data (i.e. animal position, positions of the target areas, etc.) from each cycle (video frame) of the experiment will be saved. The Excel-file ("ExperimentID_protocol.xlsx") will be saved to the folder containing this notebook. 

In [ ]:
# Create an Excel workbook and worksheet
protocolName  = experimentID + '_protocol.xlsx'
workbook = xlsxwriter.Workbook(protocolName, {'constant_memory': True, 'tmpdir': './'})

# Workbook = xlsxwriter.Workbook(protocolName)
worksheet = workbook.add_worksheet()

# First row of the Excel sheet with column headings
protocolRow = (
        ['FrameID', 'Time [s]', 'Phase', 'Animal_x', 'Animal_y', 'Start_x', 'Start_y', 'Start_rad', 'Target_x', 'Target_y', 'Target_rad', 'TrialID', 'Rewarded Trials [%]', 'Sound Played', 'Common cycle ID', 'Odd1_x', 'Odd1_Y', 'Odd2_x', 'Odd2_Y', 'Odd3_x', 'Odd3_Y', 'Odd4_x', 'Odd4_Y'],
        )
row = 0
col = 0

# Provides column headings for the protocol
for frame, timeStamp, phase, anX, anY, stX, stY, stRad, tarX, tarY, tarRad, trial, rewP, soundP, comCy, odd1X, odd1Y, odd2X, odd2Y, odd3X, odd3Y, odd4X, odd4Y in (protocolRow):
    worksheet.write(row, col,     frame)        # Frame ID
    worksheet.write(row, col + 1, timeStamp)    # Time stamp
    worksheet.write(row, col + 2, phase)        # Phase of experiment
    worksheet.write(row, col + 3, anX)          # X-Coordinate of the subject
    worksheet.write(row, col + 4, anY)          # Y-Coordinate of the subject
    worksheet.write(row, col + 5, stX)          # X-Coordinate of the starting platform
    worksheet.write(row, col + 6, stY)          # Y-Coordinate of the starting platform
    worksheet.write(row, col + 7, stRad)        # Radius of the starting platform
    worksheet.write(row, col + 8, tarX)         # X-Coordinate of the target 
    worksheet.write(row, col + 9, tarY)         # Y-Coordinate of the target 
    worksheet.write(row, col + 10, tarRad)      # Radius of the target platform
    worksheet.write(row, col + 11, trial)       # Trial ID
    worksheet.write(row, col + 12, rewP)        # Percentage of trials rewarded
    worksheet.write(row, col + 13, soundP)      # sound played
    worksheet.write(row, col + 14, comCy)       # common cycle ID
    worksheet.write(row, col + 15, odd1X)       # X-Coordinate of the odd1 target 
    worksheet.write(row, col + 16, odd1Y)       # Y-Coordinate of the odd1 target 
    worksheet.write(row, col + 17, odd2X)       # X-Coordinate of the odd2 target 
    worksheet.write(row, col + 18, odd2Y)       # Y-Coordinate of the odd2 target 
    worksheet.write(row, col + 19, odd3X)       # X-Coordinate of the odd3 target 
    worksheet.write(row, col + 20, odd3Y)       # Y-Coordinate of the odd3 target 
    worksheet.write(row, col + 21, odd4X)       # X-Coordinate of the odd4 target 
    worksheet.write(row, col + 22, odd4Y)       # Y-Coordinate of the odd4 target 
    row += 1


## Open a start button

This cell provides a start button. If you run this notebook cell-by-cell, this button is obsolete. However, if you run all cells at once, this is the point of no return. Once you have started the experiment, it cannot be paused until the session criteria are met or it is interrupted manually.

In [ ]:
root = tk.Tk()
frame = tk.Frame(root)
frame.pack()
button = tk.Button(frame, 
                   text="Start Experiment!", 
                   fg="black",
                   command=root.destroy)
button.pack(side=tk.LEFT)
#def abs():
#    root.destroy

root.mainloop()



## Start the experiment

The final cell contains all the code for animal tracking and hardware control in response to the animals's behavior. We hope that the comments provided in the code suffice to understand the individual steps and to adjust them to your own setup and needs, if necessary.

The experiment will stop automatically, if either one of the following conditions is met:

(1) The pre-defined session duration is reached; <br/>
(2) The pre-definde number of trials is reached; <br/>
(3) The experiment is voluntarily stopped prematurely by hitting "q". 

If you should decide to stop the experiment manually, always use the "q"-button on your keyboard. Just quitting Jupyter/Python will lead to data loss!

In [ ]:
# Define and start the experiment timer
expTime = time.time()

# Start the audio stream
stream.start()

# Conditions to be met for the experiment to start and continue
while(cap.isOpened() and trialCounter<trialNumber and (time.time()-expTime)<=sessionDuration):
    
    # Here you can choose different modes of amplitude modulation by commenting/uncommenting 
    ampMod = (random.randrange(2396,2962,1)/100)**e/10000 # Unbiased Voltage Ratio -5dB
    ### ampMod = random.randrange(5623,10001,1)/10000 # Voltage Ratio -5dB
    ### ampMod = random.randrange(3162,10001,1)/10000 # Power Ratio -5dB
    ### ampMod = 1 # No modulation
    
    # Phase 0 = Animal just entered the arena or finished a trial
    if expPhase == 0:
    
        ret, frame = cap.read()

        if ret==True:

            maskedFrame = cv2.bitwise_and(src1 = frame, src2 = mask)
            
            # In phase 0, there is no acoustic stimulation, so this is 
            # kept at 1 for all cycles spent in expPhase 0
            commonCycle = 1
            
            # Animal tracking
            # Substracts background from current frame
            if backgroundColor == 'T':
                subject = cv2.subtract(img,maskedFrame)
            else:
                subject = cv2.subtract(maskedFrame,img)
    
            # Converts subject to grey scale
            subjectGray = cv2.cvtColor(subject, cv2.COLOR_BGR2GRAY)
            
            # Applies blur and thresholding to the subject
            kernelSize = (25,25)
            frameBlur = cv2.GaussianBlur(subjectGray, kernelSize, 0)
            _, thresh = cv2.threshold(frameBlur, 40, 255, cv2.THRESH_BINARY)
            
            
            # Finds contours and selects the contour with the largest area
            ###contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            _, contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            
            # If there is no subject, the sreen is blackened, indicating that there is a problem
            # with the tracking or that your animal has escaped.
            # This code block helps when building and testing the setup. During a real experiment,
            # the condition hopefully is never met. 
            if (len(contours) == 0): 
                x = 20
                y = 40
                subjectHullCentroid = np.zeros(frame.shape,np.uint8)
                subjectHullCentroid = cv2.circle(subjectHullCentroid, (x,y), 3, BGR_COLOR['yellow'], -1)
            
            # If there is a subject, it is tracked
            else:
                contour = contours[np.argmax(list(map(cv2.contourArea, contours)))]
                M = cv2.moments(contour)
                if ((M['m00']) == 0):
                    x = 20
                    y = 40
                    subjectHullCentroid = np.zeros(frame.shape,np.uint8)
                    subjectHullCentroid = cv2.circle(subjectHullCentroid, (x,y), 3, BGR_COLOR['yellow'], -1)
                else:
                    x = int(M['m10'] / M['m00'])
                    y = int(M['m01'] / M['m00'])
                    hull = cv2.convexHull(contour)
                    subjectHullCentroid = maskedFrame
          
                # Draws contour and centroid of the subject
                cv2.drawContours(subjectHullCentroid, [contour], 0, BGR_COLOR['green'], 1, cv2.LINE_AA)
                subjectHullCentroid = cv2.circle(subjectHullCentroid, (x,y), 3, BGR_COLOR['yellow'], -1)
            
            # Draws the arena contour, the starting platform, and a red dot, signalling that the subject is outside the starting area
            subjectHullCentroidArena = cv2.circle(subjectHullCentroid,(arenaX,arenaY), arenaRadius, (0,0,255), 2)
            subjectHullCentroidArenaStart = cv2.circle(subjectHullCentroidArena,(startX,startY), startRadius, (255,0,255), 2)
            subjectHullCentroidArenaStartOut = cv2.circle(subjectHullCentroidArena,(20,20), 10, BGR_COLOR['red'], -6)
 
            # Adds a stopwatch for the experiment duration to the video
            subjectHullCentroidArenaStartOutText=cv2.putText(subjectHullCentroidArenaStartOut,
            '' + str('Time: %.2f' % ((time.time()-expTime))),
            (10,590), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['white'])
            
            # Adds the current trial number to the video
            subjectHullCentroidArenaStartOutText=cv2.putText(subjectHullCentroidArenaStartOutText,
            '' + str('Trial#: %.0f' % (trialCounter)),
            (670,30), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['blue'])
            
            # Adds the current number of collected rewards to the video
            subjectHullCentroidArenaStartOutText=cv2.putText(subjectHullCentroidArenaStartOutText,
            '' + str('Reward#: %.0f' % (rewardCounter)),
            (670,50), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['blue'])
            
            # Writes the modified frame to the video protocol and shows it in a popup window 
            out.write(subjectHullCentroidArenaStartOutText)
            cv2.imshow('Press (q)-to end the experiment',subjectHullCentroidArenaStartOutText)

            # Frame ID
            frameCounter = frameCounter+1
            
            # Calculates the percentage of successful/rewarded trials
            if (rewardCounter==0):
                percentCorrect = 0
            else:
                percentCorrect = 100/trialCounter*rewardCounter
            
            # Feede an empty wave to the audio stream
            stream.write(silenceSound)
            soundPlayed = 'false'
            
            # Writes a new row to the Excel-protocol
            protocolRow = (
                    [frameCounter, (time.time()-expTime), expPhase, x, y, startX, startY, startRadius, targetX, targetY, targetRadius, trialCounter, percentCorrect, soundPlayed, commonCycle, target2X, target2Y, target3X, target3Y, target4X, target4Y, target5X, target5Y],
                    )
            for frame, timeStamp, phase, anX, anY, stX, stY, stRad, tarX, tarY, tarRad, trial, rewP, soundP, comCy, odd1X, odd1Y, odd2X, odd2Y, odd3X, odd3Y, odd4X, odd4Y in (protocolRow):
                worksheet.write(row, col,     frame)
                worksheet.write(row, col + 1, timeStamp)
                worksheet.write(row, col + 2, phase)
                worksheet.write(row, col + 3, anX)
                worksheet.write(row, col + 4, anY)
                worksheet.write(row, col + 5, stX)
                worksheet.write(row, col + 6, stY)
                worksheet.write(row, col + 7, stRad)
                worksheet.write(row, col + 8, tarX)
                worksheet.write(row, col + 9, tarY)
                worksheet.write(row, col + 10, tarRad)
                worksheet.write(row, col + 11, trial)
                worksheet.write(row, col + 12, rewP)  
                worksheet.write(row, col + 13, soundP)      
                worksheet.write(row, col + 14, comCy)       
                worksheet.write(row, col + 15, odd1X)  
                worksheet.write(row, col + 16, odd1Y)    
                worksheet.write(row, col + 17, odd2X)    
                worksheet.write(row, col + 18, odd2Y)    
                worksheet.write(row, col + 19, odd3X)     
                worksheet.write(row, col + 20, odd3Y)    
                worksheet.write(row, col + 21, odd4X)  
                worksheet.write(row, col + 22, odd4Y)  
                row += 1

            # Checks, if the subject is in the starting/initialization area
            # If so, the protocol proceeds to phase 1 and a timer is started
            if (((x-startX)*(x-startX))+((y-startY)*(y-startY))) <= (startRadius*startRadius):
                expPhase = 1
                startInZone = time.time()
            
            # If not, the protocol remains in phase 0
            else:
                expPhase = 0
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break          
  
        else:
            break
    
    # Phase 1 = Animal is in the starting area
    elif expPhase == 1:
    
        ret, frame = cap.read()

        if ret==True:

            maskedFrame = cv2.bitwise_and(src1 = frame, src2 = mask)
                    
            ## Animal tracking
            # Substracts background from current frame
            if backgroundColor == 'T':
                subject = cv2.subtract(img,maskedFrame)
            else:
                subject = cv2.subtract(maskedFrame,img)
      
            # Converts subject to grey scale
            subjectGray = cv2.cvtColor(subject, cv2.COLOR_BGR2GRAY)
            
            # Applies blur and thresholding to the subject
            kernelSize = (25,25)
            frameBlur = cv2.GaussianBlur(subjectGray, kernelSize, 0)
            _, thresh = cv2.threshold(frameBlur, 40, 255, cv2.THRESH_BINARY)
            
            # Finds contours and selects the contour with the largest area
            ###contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            _, contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            
            # If there is no subject, the sreen is blackened, indicating that there is a problem
            # with the tracking or that your animal has escaped.
            # This code block helps when building and testing the setup. During a real experiment,
            # the condition hopefully is never met. 
            if (len(contours) == 0):
                x = 20
                y = 40
                subjectHullCentroid = np.zeros(frame.shape,np.uint8)
                subjectHullCentroid = cv2.circle(subjectHullCentroid, (x,y), 3, BGR_COLOR['yellow'], -1)

            # If there is a subject, it is tracked
            else:
                contour = contours[np.argmax(list(map(cv2.contourArea, contours)))]
                M = cv2.moments(contour)
                if ((M['m00']) == 0):
                    x = 20
                    y = 40
                    subjectHullCentroid = np.zeros(frame.shape,np.uint8)
                    subjectHullCentroid = cv2.circle(subjectHullCentroid, (x,y), 3, BGR_COLOR['yellow'], -1)
                else:
                    x = int(M['m10'] / M['m00'])
                    y = int(M['m01'] / M['m00'])
                    hull = cv2.convexHull(contour)
                    subjectHullCentroid = maskedFrame
          
            # Draws contour and centroid of the subject
            cv2.drawContours(subjectHullCentroid, [contour], 0, BGR_COLOR['green'], 1, cv2.LINE_AA)
            subjectHullCentroid = cv2.circle(subjectHullCentroid, (x,y), 3, BGR_COLOR['yellow'], -1)
            
            # Draws the arena contour, the starting platform, and a green dot, signalling that the subject is inside the starting area
            subjectHullCentroidArena = cv2.circle(subjectHullCentroid,(arenaX,arenaY), arenaRadius, (0,0,255), 2)
            subjectHullCentroidArenaStart = cv2.circle(subjectHullCentroidArena,(startX,startY), startRadius, (255,0,255), 2)
            subjectHullCentroidArenaStartIn = cv2.circle(subjectHullCentroidArena,(20,20), 10, BGR_COLOR['green'], -6)
            
            # Adds a stopwatch for the experiment duration to the video
            subjectHullCentroidArenaStartInText=cv2.putText(subjectHullCentroidArenaStartIn,
            '' + str('Time: %.2f' % ((time.time()-expTime))),
            (10,590), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['white'])   
            
            # Adds the current trial number to the video
            subjectHullCentroidArenaStartInText=cv2.putText(subjectHullCentroidArenaStartInText,
            '' + str('Trial#: %.0f' % (trialCounter)),
            (670,30), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['blue'])
            
            # Adds the current number of collected rewards to the video
            subjectHullCentroidArenaStartInText=cv2.putText(subjectHullCentroidArenaStartInText,
            '' + str('Reward#: %.0f' % (rewardCounter)),
            (670,50), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['blue'])
        
            # Writes the modified frame to the video protocol and shows it in a popup window 
            out.write(subjectHullCentroidArenaStartInText)
            cv2.imshow('Press (q)-to end the experiment',subjectHullCentroidArenaStartInText)
                        
            # Frame ID
            frameCounter = frameCounter+1
            
            # Calculates the percentage of successful/rewarded trials
            if (rewardCounter==0):
                percentCorrect = 0
            else:
                percentCorrect = 100/trialCounter*rewardCounter
            
            ## Checks, if the subject is still in the starting/initialization area 
            if (((x-startX)*(x-startX))+((y-startY)*(y-startY))) <= (startRadius*startRadius):
                stopInZone = time.time()
                # Checks, if the time spent in the starting/initialization area exceeds the initiation duration
                # If so, the protocol proceeds to phase 2, the trial timer is started, the designated distractor (trial)
                # sound is played every "delayLength" cycles, and the target areas for the current trial are generated
                if (stopInZone-startInZone) >= initDuration:
                    expPhase = 2
                    startTrial = time.time()
                    if (commonCycle == 1):
                        stream.write((distractorSoundTrial*ampMod))
                        commonCycle = commonCycle+1 
                        soundPlayed = 'true-DistractorTrial'
                    elif (commonCycle < delayLength and commonCycle >= 2):
                        stream.write(silenceSound)
                        commonCycle = commonCycle+1
                        soundPlayed = 'false'
                    elif (commonCycle == delayLength):
                        stream.write(silenceSound)
                        commonCycle = 1
                        soundPlayed = 'false'
                    
                    # Generates the first target (attractor), which cannot overlap with the starting area
                    while ((((randomX-arenaX)*(randomX-arenaX))+((randomY-arenaY)*(randomY-arenaY))) >= (arenaRadius*arenaRadius) or               
                               math.sqrt(((startX-randomX)*(startX-randomX))+((startY-randomY)*(startY-randomY))) <= (startRadius+targetRadius)):  

                            # random angle
                            alpha = 2 * math.pi * random.random()   
                            # random radius
                            r = (arenaRadius-20-targetRadius) * math.sqrt(random.random())  
                            # calculating coordinates
                            randomX = int(r * math.cos(alpha) + arenaX)  
                            randomY = int(r * math.sin(alpha) + arenaY)  
                            targetX = randomX                            
                            targetY = randomY                      
         
                # If the duration spent in the starting area does not exceed the initialization duration, 
                # the protocol remains in phase 1
                else:
                    stream.write(silenceSound)
                    soundPlayed = 'false'
                    commonCycle = 1
                    expPhase = 1
            
            # If the animal leaves the starting area before the initialization duration is reached, 
            # the protocol goes back to phase 0
            else:   
                stream.write(silenceSound)
                soundPlayed = 'false'
                commonCycle = 1
                expPhase = 0
 
            # Writes a new row to the Excel-protocol
            protocolRow = (
                    [frameCounter, (time.time()-expTime), expPhase, x, y, startX, startY, startRadius, targetX, targetY, targetRadius, trialCounter, percentCorrect, soundPlayed, commonCycle, target2X, target2Y, target3X, target3Y, target4X, target4Y, target5X, target5Y],
                    )
            for frame, timeStamp, phase, anX, anY, stX, stY, stRad, tarX, tarY, tarRad, trial, rewP, soundP, comCy, odd1X, odd1Y, odd2X, odd2Y, odd3X, odd3Y, odd4X, odd4Y in (protocolRow):
                worksheet.write(row, col,     frame)
                worksheet.write(row, col + 1, timeStamp)
                worksheet.write(row, col + 2, phase)
                worksheet.write(row, col + 3, anX)
                worksheet.write(row, col + 4, anY)
                worksheet.write(row, col + 5, stX)
                worksheet.write(row, col + 6, stY)
                worksheet.write(row, col + 7, stRad)
                worksheet.write(row, col + 8, tarX)
                worksheet.write(row, col + 9, tarY)
                worksheet.write(row, col + 10, tarRad)
                worksheet.write(row, col + 11, trial)
                worksheet.write(row, col + 12, rewP)    
                worksheet.write(row, col + 13, soundP)      
                worksheet.write(row, col + 14, comCy)                    
                worksheet.write(row, col + 15, odd1X)  
                worksheet.write(row, col + 16, odd1Y)    
                worksheet.write(row, col + 17, odd2X)    
                worksheet.write(row, col + 18, odd2Y)    
                worksheet.write(row, col + 19, odd3X)     
                worksheet.write(row, col + 20, odd3Y)    
                worksheet.write(row, col + 21, odd4X)  
                worksheet.write(row, col + 22, odd4Y)     
                row += 1    
        
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break          
  
        else:
            break
    
    ########################
    # Phase 2 = Animal initiated the trial
    elif expPhase == 2:
        
        ret, frame = cap.read()
        if ret==True:

            maskedFrame = cv2.bitwise_and(src1 = frame, src2 = mask)
            
            ## Animal tracking
            # Substracts background from current frame
            if backgroundColor == 'T':
                subject = cv2.subtract(img,maskedFrame)
            else:
                subject = cv2.subtract(maskedFrame,img)
       
            # Converts subject to grey scale
            subjectGray = cv2.cvtColor(subject, cv2.COLOR_BGR2GRAY)
            
            # Applies blur and thresholding to the subject
            kernelSize = (25,25)
            frameBlur = cv2.GaussianBlur(subjectGray, kernelSize, 0)
            _, thresh = cv2.threshold(frameBlur, 40, 255, cv2.THRESH_BINARY)
            
            # Finds contours and selects the contour with the largest area
            ###contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            _, contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            
            # If there is no subject, the sreen is blackened, indicating that there is a problem
            # with the tracking or that your animal has escaped.
            # This code block helps when building and testing the setup. During a real experiment,
            # the condition hopefully is never met. 
            if (len(contours) == 0):
                x = 20
                y = 40
                subjectHullCentroid = np.zeros(frame.shape,np.uint8)
                subjectHullCentroid = cv2.circle(subjectHullCentroid, (x,y), 3, BGR_COLOR['yellow'], -1)

            # If there is a subject, it is tracked
            else:
            
                contour = contours[np.argmax(list(map(cv2.contourArea, contours)))]
                M = cv2.moments(contour)
                if ((M['m00']) == 0):
                    x = 20
                    y = 40
                    subjectHullCentroid = np.zeros(frame.shape,np.uint8)
                    subjectHullCentroid = cv2.circle(subjectHullCentroid, (x,y), 3, BGR_COLOR['yellow'], -1)
                else:
                    x = int(M['m10'] / M['m00'])
                    y = int(M['m01'] / M['m00'])
                    hull = cv2.convexHull(contour)
                    subjectHullCentroid = maskedFrame
      
            # Draws contour and centroid of the subject
            cv2.drawContours(subjectHullCentroid, [contour], 0, BGR_COLOR['green'], 1, cv2.LINE_AA)
            subjectHullCentroid = cv2.circle(subjectHullCentroid, (x,y), 3, BGR_COLOR['yellow'], -1)
            
            # Draws the arena contour, the attractor target, and a blue dot, 
            # signalling that the subject is outside the attractor target area            
            subjectHullCentroidArena = cv2.circle(subjectHullCentroid,(arenaX,arenaY), arenaRadius, (0,0,255), 2)
            subjectHullCentroidArenaStart = cv2.circle(subjectHullCentroidArena,(targetX,targetY), targetRadius, (0,255,0), 2)
            subjectHullCentroidArenaStartIn = cv2.circle(subjectHullCentroidArena,(20,20), 10, BGR_COLOR['blue'], -6)

            # Adds a stopwatch for the experiment duration to the video            
            subjectHullCentroidArenaStartInText=cv2.putText(subjectHullCentroidArenaStartIn,
            '' + str('Time: %.2f' % ((time.time()-expTime))),
            (10,590), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['white'])
            
            # Adds a trial duration countdown to the video
            subjectHullCentroidArenaStartInText=cv2.putText(subjectHullCentroidArenaStartInText,
            '' + str('Trial: %.2f' % ((trialDuration-trialCountdown))),
            (670,590), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['red'])

            # Adds the current trial number to the video
            subjectHullCentroidArenaStartInText=cv2.putText(subjectHullCentroidArenaStartInText,
            '' + str('Trial#: %.0f' % (trialCounter)),
            (670,30), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['blue'])
            
            # Adds the current number of collected rewards to the video
            subjectHullCentroidArenaStartInText=cv2.putText(subjectHullCentroidArenaStartInText,
            '' + str('Reward#: %.0f' % (rewardCounter)),
            (670,50), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['blue'])
            
            # Writes the modified frame to the video protocol and shows it in a popup window 
            out.write(subjectHullCentroidArenaStartInText)
            cv2.imshow('Press (q)-to end the experiment',subjectHullCentroidArenaStartInText)
            
            # Frame ID
            frameCounter = frameCounter+1            
            
            # Calculates the percentage of successful/rewarded trials
            if (rewardCounter==0):
                percentCorrect = 0
            else:
                percentCorrect = 100/trialCounter*rewardCounter

            # Current time               
            stopTrial = time.time()
            
            # If the maximum trial duration is reached, the trial is terminated and the protocol goes back to phase 0
            if (stopTrial-startTrial) >= trialDuration:
                expPhase=0
                trialCounter = trialCounter+1
                randomX = 9000
                randomY = 9000            
                random2X = 9000
                random2Y = 9000                
                random3X = 9000
                random3Y = 9000
                random4X = 9000
                random4Y = 9000
                random5X = 9000
                random5Y = 9000        
                trialCountdown = 0
                
            else:
                
                # Time left for trial successful trial completion
                trialCountdown = (stopTrial-startTrial)
                
                # Checks, if the animal is in the attractor target area
                # If so, acoustic stimulation switches to the designated attractor stimulus and the protocol 
                # proceeds to phase 3
                if (((x-targetX)*(x-targetX))+((y-targetY)*(y-targetY))) <= (targetRadius*targetRadius):
                    startInTarget = time.time()
                    if (commonCycle == 1):
                        stream.write((attractorSoundTarget1*ampMod)) 
                        commonCycle = commonCycle+1
                        stopInTarget = time.time()
                        #print((stopInTarget-startInTarget))
                        soundPlayed = 'true-AttractorTarget1'
                        expPhase = 3
                    elif (commonCycle < delayLength and commonCycle >= 2):
                        stream.write(silenceSound)
                        commonCycle = commonCycle+1
                        stopInTarget = time.time()
                        #print((stopInTarget-startInTarget))
                        soundPlayed = 'false'
                        expPhase = 3
                    elif (commonCycle == delayLength):
                        stream.write(silenceSound)
                        commonCycle = 1
                        stopInTarget = time.time()
                        #print((stopInTarget-startInTarget))
                        soundPlayed = 'false'
                        expPhase = 3
                
                # If the animal is not in the target areas, the protocol keeps playing back the designated trial
                # distractor stimulus and remains in phase 2
                else:
                    if (commonCycle == 1):
                        stream.write((distractorSoundTrial*ampMod))
                        soundPlayed = 'true-DistractorTrial'
                        commonCycle = commonCycle+1 
                        expPhase = 2
                    elif (commonCycle < delayLength and commonCycle >= 2):
                        stream.write(silenceSound)
                        soundPlayed = 'false'
                        commonCycle = commonCycle+1
                        expPhase = 2
                    elif (commonCycle == delayLength):
                        stream.write(silenceSound)
                        soundPlayed = 'false'
                        commonCycle = 1
                        expPhase = 2
                            
            # Writes a new row to the Excel-protocol        
            protocolRow = (
                    [frameCounter, (time.time()-expTime), expPhase, x, y, startX, startY, startRadius, targetX, targetY, targetRadius, trialCounter, percentCorrect, soundPlayed, commonCycle, target2X, target2Y, target3X, target3Y, target4X, target4Y, target5X, target5Y],
                    )
            for frame, timeStamp, phase, anX, anY, stX, stY, stRad, tarX, tarY, tarRad, trial, rewP, soundP, comCy, odd1X, odd1Y, odd2X, odd2Y, odd3X, odd3Y, odd4X, odd4Y in (protocolRow):
                worksheet.write(row, col,     frame)
                worksheet.write(row, col + 1, timeStamp)
                worksheet.write(row, col + 2, phase)
                worksheet.write(row, col + 3, anX)
                worksheet.write(row, col + 4, anY)
                worksheet.write(row, col + 5, stX)
                worksheet.write(row, col + 6, stY)
                worksheet.write(row, col + 7, stRad)
                worksheet.write(row, col + 8, tarX)
                worksheet.write(row, col + 9, tarY)
                worksheet.write(row, col + 10, tarRad)
                worksheet.write(row, col + 11, trial)
                worksheet.write(row, col + 12, rewP)
                worksheet.write(row, col + 13, soundP)      
                worksheet.write(row, col + 14, comCy)      
                worksheet.write(row, col + 15, odd1X)  
                worksheet.write(row, col + 16, odd1Y)    
                worksheet.write(row, col + 17, odd2X)    
                worksheet.write(row, col + 18, odd2Y)    
                worksheet.write(row, col + 19, odd3X)     
                worksheet.write(row, col + 20, odd3Y)    
                worksheet.write(row, col + 21, odd4X)  
                worksheet.write(row, col + 22, odd4Y)  
                row += 1
                    
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break     

        else:
            break       
        
    # Phase 3 = Animal entered the target area
    elif expPhase == 3:
    
        ret, frame = cap.read()
        if ret==True:

            maskedFrame = cv2.bitwise_and(src1 = frame, src2 = mask)
            
            ## Animal tracking
            # Substracts background from current frame
            if backgroundColor == 'T':
                subject = cv2.subtract(img,maskedFrame)
            else:
                subject = cv2.subtract(maskedFrame,img)
        
            # Converts subject to grey scale
            subjectGray = cv2.cvtColor(subject, cv2.COLOR_BGR2GRAY)
            
            # Applies blur and thresholding to the subject
            kernelSize = (25,25)
            frameBlur = cv2.GaussianBlur(subjectGray, kernelSize, 0)
            _, thresh = cv2.threshold(frameBlur, 40, 255, cv2.THRESH_BINARY)
            
            # Finds contours and selects the contour with the largest area
            ###contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            _, contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
          
            # If there is no subject, the sreen is blackened, indicating that there is a problem
            # with the tracking or that your animal has escaped.
            # This code block helps when building and testing the setup. During a real experiment,
            # the condition hopefully is never met. 
            if (len(contours) == 0):
                x = 20
                y = 40
                subjectHullCentroid = np.zeros(frame.shape,np.uint8)
                subjectHullCentroid = cv2.circle(subjectHullCentroid, (x,y), 3, BGR_COLOR['yellow'], -1)

            # If there is a subject, it is tracked
            else:
                contour = contours[np.argmax(list(map(cv2.contourArea, contours)))]
                M = cv2.moments(contour)
                if ((M['m00']) == 0):
                    x = 780
                    y = 580
                    subjectHullCentroid = np.zeros(frame.shape,np.uint8)
                    subjectHullCentroid = cv2.circle(subjectHullCentroid, (x,y), 3, BGR_COLOR['yellow'], -1)
                else:
                    x = int(M['m10'] / M['m00'])
                    y = int(M['m01'] / M['m00'])
                    hull = cv2.convexHull(contour)
                    subjectHullCentroid = maskedFrame
                
          
            # Draws contour and centroid of the subject
            cv2.drawContours(subjectHullCentroid, [contour], 0, BGR_COLOR['green'], 1, cv2.LINE_AA)
            subjectHullCentroid = cv2.circle(subjectHullCentroid, (x,y), 3, BGR_COLOR['yellow'], -1)
            
            # Draws the arena contour, the attractor target, the distractor target, and a green dot, 
            # signalling that the subject is inside the attractor target area
            subjectHullCentroidArena = cv2.circle(subjectHullCentroid,(arenaX,arenaY), arenaRadius, (0,0,255), 2)
            subjectHullCentroidArenaStart = cv2.circle(subjectHullCentroidArena,(targetX,targetY), targetRadius, (0,255,0), 2)
            subjectHullCentroidArenaStartIn = cv2.circle(subjectHullCentroidArena,(20,20), 10, BGR_COLOR['green'], -6)
      
            # Adds a stopwatch for the experiment duration to the video
            subjectHullCentroidArenaStartInText=cv2.putText(subjectHullCentroidArenaStartIn,
            '' + str('Time: %.2f' % ((time.time()-expTime))),
            (10,590), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['white'])
            
            # Adds a trial duration countdown to the video
            subjectHullCentroidArenaStartInText=cv2.putText(subjectHullCentroidArenaStartInText,
            '' + str('Trial: %.2f' % ((trialDuration-trialCountdown))),
            (670,590), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['green'])
            
            # Adds a target duration countdown to the video
            subjectHullCentroidArenaStartInText=cv2.putText(subjectHullCentroidArenaStartInText,
            '' + str('Target: %.2f' % ((targetDuration-targetCountdown))),
            (670,570), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['green'])
     
            # Adds the current trial number to the video
            subjectHullCentroidArenaStartInText=cv2.putText(subjectHullCentroidArenaStartInText,
            '' + str('Trial#: %.0f' % (trialCounter)),
            (670,30), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['blue'])
            
            # Adds the current number of collected rewards to the video
            subjectHullCentroidArenaStartInText=cv2.putText(subjectHullCentroidArenaStartInText,
            '' + str('Reward#: %.0f' % (rewardCounter)),
            (670,50), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['blue'])
            
            # Writes the modified frame to the video protocol and shows it in a popup window 
            out.write(subjectHullCentroidArenaStartInText)
            cv2.imshow('Press (q)-to end the experiment',subjectHullCentroidArenaStartInText)
            
            # Frame ID
            frameCounter = frameCounter+1
            
            # Calculates the percentage of successful/rewarded trials
            if (rewardCounter==0):
                percentCorrect = 0
            else:
                percentCorrect = 100/trialCounter*rewardCounter
            
            # Checks, if the animal is still in the attractor target area
            # If so, acoustic stimulation continues with the designated attractor stimulus and the protocol 
            # remains in phase 3          
            if (((x-targetX)*(x-targetX))+((y-targetY)*(y-targetY))) <= (targetRadius*targetRadius):
                
                if (commonCycle == 1):
                    stream.write((attractorSoundTarget1*ampMod)) 
                    stopInTarget = time.time()
                    soundPlayed = 'true-AttractorTarget1'
                    commonCycle = commonCycle+1
                    expPhase = 3
                elif (commonCycle < delayLength and commonCycle >= 2):
                    stream.write(silenceSound)
                    stopInTarget = time.time()
                    soundPlayed = 'false'
                    commonCycle = commonCycle+1
                    expPhase = 3
                elif (commonCycle == delayLength):
                    stream.write(silenceSound)
                    stopInTarget = time.time()
                    soundPlayed = 'false'
                    commonCycle = 1
                    expPhase = 3
                    
                # Checks, if the desired target duration is reached   
                # If so, the subject is rewarded, the trial and reward counters are increased by 1,
                # the target countdown stops, and the protocol goes back to phase 1
                if (stopInTarget-startInTarget) >= targetDuration:
                    trialCounter = trialCounter+1
                    randomX = 9000
                    randomY = 9000
                    rewardCounter = rewardCounter+1
                    targetCountdown = 0
                    
                    # Activates the current feeder and switches to the other feeder for the next reward
                    if (feederID == 1):
                        while True:
                            try:
                                board.digital[arduinoFeeder1].write(1) 
                                time.sleep(.068)
                                board.digital[arduinoFeeder1].write(0) 
                                startTrial = time.time()
                                feederID = 2
                                expPhase = 0
                            # This code block helps when building and testing the setup. During a real experiment,
                            # the condition hopefully is never met.
                            except:
                                decision = input("Feeder Error: "
                                                 "Please check the Arduino for problems! "
                                                 "Try to continue with reward? [Y]: ")
                                if decision == 'Y':
                                    continue
                                elif decision != 'Y':
                                    startTrial = time.time()
                                    feederID = 2
                                    expPhase = 0
                                    break
                            break
                        
                    # Activates the current feeder and switches to the other feeder for the next reward   
                    elif (feederID == 2):
                        while True:
                            try:
                                board.digital[arduinoFeeder2].write(1) 
                                time.sleep(.068)
                                board.digital[arduinoFeeder2].write(0) 
                                startTrial = time.time()
                                feederID = 1
                                expPhase = 0
                            # This code block helps when building and testing the setup. During a real experiment,
                            # the condition hopefully is never met.
                            except:
                                decision = input("Feeder Error: "
                                                 "Please check the Arduino for problems! "
                                                 "Try to continue with reward? [Y]: ")
                                if decision == 'Y':
                                    continue
                                elif decision != 'Y':
                                    startTrial = time.time()
                                    feederID = 1
                                    expPhase = 0
                                    break
                            break
                
                # If the desired target duration is not reached, the protocol remains in phase 3 and the
                # countdown continues
                else:
                    expPhase = 3
                    targetCountdown = (stopInTarget-startInTarget)

            # If the animal has left the attractor target area, the protocol switches to the designated trial
            # distractor stimulus and goes back to phase 2
            else:
                if (commonCycle == 1):
                    stream.write((distractorSoundTrial*ampMod))
                    soundPlayed = 'true-DistractorTrial'
                    commonCycle = commonCycle+1 
                    expPhase = 2
                elif (commonCycle < delayLength and commonCycle >= 2):
                    stream.write(silenceSound)
                    soundPlayed = 'false'
                    commonCycle = commonCycle+1
                    expPhase = 2
                elif (commonCycle == delayLength):
                    stream.write(silenceSound)
                    soundPlayed = 'false'
                    commonCycle = 1
                    expPhase = 2

            # Writes a new row to the Excel-protocol
            protocolRow = (
                    [frameCounter, (time.time()-expTime), expPhase, x, y, startX, startY, startRadius, targetX, targetY, targetRadius, trialCounter, percentCorrect, soundPlayed, commonCycle, target2X, target2Y, target3X, target3Y, target4X, target4Y, target5X, target5Y],
                    )
            for frame, timeStamp, phase, anX, anY, stX, stY, stRad, tarX, tarY, tarRad, trial, rewP, soundP, comCy, odd1X, odd1Y, odd2X, odd2Y, odd3X, odd3Y, odd4X, odd4Y in (protocolRow):
                worksheet.write(row, col,     frame)
                worksheet.write(row, col + 1, timeStamp)
                worksheet.write(row, col + 2, phase)
                worksheet.write(row, col + 3, anX)
                worksheet.write(row, col + 4, anY)
                worksheet.write(row, col + 5, stX)
                worksheet.write(row, col + 6, stY)
                worksheet.write(row, col + 7, stRad)
                worksheet.write(row, col + 8, tarX)
                worksheet.write(row, col + 9, tarY)
                worksheet.write(row, col + 10, tarRad)
                worksheet.write(row, col + 11, trial)
                worksheet.write(row, col + 12, rewP)
                worksheet.write(row, col + 13, soundP)      
                worksheet.write(row, col + 14, comCy)                       
                worksheet.write(row, col + 15, odd1X)  
                worksheet.write(row, col + 16, odd1Y)    
                worksheet.write(row, col + 17, odd2X)    
                worksheet.write(row, col + 18, odd2Y)    
                worksheet.write(row, col + 19, odd3X)     
                worksheet.write(row, col + 20, odd3Y)    
                worksheet.write(row, col + 21, odd4X)  
                worksheet.write(row, col + 22, odd4Y)  
                row += 1
                
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break          
  
        else:
            break

# If the session is over or interrupted, all capture and output devices are released, streams are stopped, 
# windows are destroyed, Excel-files are saved, and the communication with the Arduino is terminated
cap.release()
out.release()
stream.stop()
cv2.destroyAllWindows()
workbook.close()
board.exit()                              

